In [1]:
!pip install transformers
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from transformers import BertTokenizer, BertModel

In [3]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [4]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [5]:
count_vectorizer = CountVectorizer()
tfidf_transformer = TfidfTransformer()

In [6]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [7]:
import torch
import numpy as np

In [18]:
nltk.download('wordnet')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
!pip install -q kaggle

In [10]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"leannalhashishi","key":"0213e501d349c1ea30f3d6c5c4e6eef1"}'}

In [11]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content"

In [12]:
!kaggle datasets download -d datasnaek/mbti-type

Dataset URL: https://www.kaggle.com/datasets/datasnaek/mbti-type
License(s): CC0-1.0
 57% 14.0M/24.4M [00:00<00:00, 52.3MB/s]
100% 24.4M/24.4M [00:00<00:00, 71.5MB/s]


In [13]:
import zipfile

with zipfile.ZipFile('mbti-type.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/mbti')

In [14]:
import pandas as pd

# load the MBTI dataset
df = pd.read_csv('/content/mbti/mbti_1.csv')
df.head()

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...


In [15]:

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))


mbti_terms = {'infj', 'intj', 'entp', 'enfp', 'istj', 'isfj', 'estp', 'esfp',
              'istp', 'isfp', 'estj', 'esfj', 'intp', 'infp', 'entj', 'enfj'}

In [16]:
def clean_text(text):
  text = text.lower() # convert text to lower case
  text = re.sub(r'http\S+|www\S+', '', text) # remove URLs
  text = re.sub(r'[^a-z\s]', ' ', text) # remove special characters
  tokens = [lemmatizer.lemmatize(word) for word in text.split()
              if word not in stop_words and word not in mbti_terms]
  return " ".join(tokens)

In [19]:
df['cleaned_posts'] = df['posts'].apply(clean_text)

In [20]:
df['cleaned_posts']

,cleaned_posts
0,moment sportscenter top ten play prank life ch...
1,finding lack post alarming sex boring position...
2,good one course say know blessing curse absolu...
3,dear enjoyed conversation day esoteric gabbing...
4,fired another silly misconception approaching ...
...,...
8670,always think cat fi doms reason website become...
8671,thread already exists someplace else heck dele...
8672,many question thing would take purple pill pic...
8673,conflicted right come wanting child honestly m...


In [21]:
X_counts = count_vectorizer.fit_transform(df['cleaned_posts'])

In [22]:
X_tfidf = tfidf_transformer.fit_transform(X_counts)

In [23]:
corpus = count_vectorizer.get_feature_names_out()

In [24]:
df['token_counts'] = list(X_counts.toarray())
df['tfidf_values'] = list(X_tfidf.toarray())

In [25]:
df['tfidf_values'][0][2000:3000]

array([0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 

In [26]:
print(df.head())

   type                                              posts  \
0  INFJ  'http://www.youtube.com/watch?v=qsXHcwe3krw|||...   
1  ENTP  'I'm finding the lack of me in these posts ver...   
2  INTP  'Good one  _____   https://www.youtube.com/wat...   
3  INTJ  'Dear INTP,   I enjoyed our conversation the o...   
4  ENTJ  'You're fired.|||That's another silly misconce...   

                                       cleaned_posts  \
0  moment sportscenter top ten play prank life ch...   
1  finding lack post alarming sex boring position...   
2  good one course say know blessing curse absolu...   
3  dear enjoyed conversation day esoteric gabbing...   
4  fired another silly misconception approaching ...   

                                        token_counts  \
0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
2  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
3  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
4  [0, 0, 

In [27]:
# add separate columns for each MBTI axis
df['IE'] = df['type'].apply(lambda x: 'I' if 'I' in x else 'E')
df['NS'] = df['type'].apply(lambda x: 'N' if 'N' in x else 'S')
df['TF'] = df['type'].apply(lambda x: 'T' if 'T' in x else 'F')
df['JP'] = df['type'].apply(lambda x: 'J' if 'J' in x else 'P')

df = df.drop(columns=['type'])
print(df.head())

                                               posts  \
0  'http://www.youtube.com/watch?v=qsXHcwe3krw|||...   
1  'I'm finding the lack of me in these posts ver...   
2  'Good one  _____   https://www.youtube.com/wat...   
3  'Dear INTP,   I enjoyed our conversation the o...   
4  'You're fired.|||That's another silly misconce...   

                                       cleaned_posts  \
0  moment sportscenter top ten play prank life ch...   
1  finding lack post alarming sex boring position...   
2  good one course say know blessing curse absolu...   
3  dear enjoyed conversation day esoteric gabbing...   
4  fired another silly misconception approaching ...   

                                        token_counts  \
0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
2  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
3  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
4  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

# Training

In [31]:
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

X = X_tfidf
axes = ['IE', 'NS', 'TF', 'JP']

param_grid = {'C': [0.1, 1, 10], 'solver': ['liblinear']}

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


In [32]:
results = {}
for axis in axes:
    print(f"Training model for {axis} axis")

    # target
    y = df[axis]

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, stratify=y, random_state=42
    )

    grid_search = GridSearchCV(
        LogisticRegression(), param_grid, cv=skf, scoring='accuracy', n_jobs=-1
    )
    grid_search.fit(X_train_tfidf, y_train)

    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test_tfidf)
    report = classification_report(y_test, y_pred)

    results[axis] = {
        'best_params': grid_search.best_params_,
        'classification_report': report
    }
    print(f"Best Params for {axis}: {grid_search.best_params_}")
    print(report)


for axis, result in results.items():
    print(f"Axis: {axis}")
    print(f"Best Parameters: {result['best_params']}")
    print("Classification Report:")
    print(result['classification_report'])
    print("\n")

Training model for IE axis...
Best Params for IE: {'C': 10, 'solver': 'liblinear'}
              precision    recall  f1-score   support

           E       0.68      0.31      0.42       400
           I       0.82      0.96      0.88      1335

    accuracy                           0.81      1735
   macro avg       0.75      0.63      0.65      1735
weighted avg       0.79      0.81      0.78      1735

Training model for NS axis...
Best Params for NS: {'C': 10, 'solver': 'liblinear'}
              precision    recall  f1-score   support

           N       0.88      0.99      0.93      1496
           S       0.78      0.16      0.26       239

    accuracy                           0.88      1735
   macro avg       0.83      0.58      0.60      1735
weighted avg       0.87      0.88      0.84      1735

Training model for TF axis...
Best Params for TF: {'C': 1, 'solver': 'liblinear'}
              precision    recall  f1-score   support

           F       0.83      0.85      0.84